In [96]:
import bw2data

In [97]:
from enbios2.generic.files import DataPath

base_folder = DataPath("temp/alex_enb2")

In [98]:
from enbios2.bw2.util import report

report()

Project: default
[]
Project: ecoinvent
['biosphere3', 'apos_3.9.1_default', 'consequential_3.9.1_default', 'cutoff_3.9.1_default', 'cutoff_3.9_default']


In [99]:
import openpyxl

excel_file = base_folder / "base_file_simplified.xlsx"
workbook = openpyxl.load_workbook(excel_file)
processors = workbook["BareProcessors simulation"]

In [100]:
# collect processor, carrier and CODE
processors_data = []
for row in processors.iter_rows(min_row=2):
    if row[1].value:
        processors_data.append((row[1].value, row[2].value, row[9].value))
# JUST TAKE THE CODES
codes = [x[2] for x in processors_data]
# COPY, IN ORDER TO CHECK WHICH ARE MISSING
remaining = processors_data.copy()

processors_data, codes

([('wind_onshore', 'electricity', '81174ec2c20931c1a36f65c654bbd11e'),
  ('wind_offshore', 'electricity', '46839adbc3cf15621ddbbe687eaa6df1'),
  ('hydro_run_of_river', 'electricity', 'b870e3d3ddd5b634d016940064b27532'),
  ('hydro_reservoir', 'electricity', 'c868c4688fbf78f5ca3787ac3d83312b'),
  ('ccgt', 'electricity', '4833b6352dfe15c95ae46fd280371cd3'),
  ('chp_biofuel_extraction',
   'electricity',
   '1952fb157a18463b0028629917057bcf'),
  ('open_field_pv', 'electricity', '206c2e2c30f45d47ea2e9a3701f8ecc5'),
  ('electrolysis', 'electricity', 'FILL'),
  ('existing_wind', 'electricity', '81174ec2c20931c1a36f65c654bbd11e'),
  ('existing_pv', 'electricity', '206c2e2c30f45d47ea2e9a3701f8ecc5'),
  ('roof_mounted_pv', 'electricity', '1d7a3591e7a3d033a25e0eaf699bb50b'),
  ('chp_wte_back_pressure', 'electricity', 'fd6356e68720aec505c69cdadb3046fc'),
  ('chp_methane_extraction',
   'electricity',
   'a0d0ab65435d0487d33b4be142483d76'),
  ('waste_supply', 'electricity', 'fd6356e68720aec505c69cd

In [101]:
bw2data.projects.set_current("ecoinvent")
list(bw2data.databases)

['biosphere3',
 'apos_3.9.1_default',
 'consequential_3.9.1_default',
 'cutoff_3.9.1_default',
 'cutoff_3.9_default']

In [102]:
from tqdm import tqdm

# read all values in column I
db = bw2data.Database("cutoff_3.9.1_default")
activities = []
missing = []
for activity in tqdm(db):
    fn = activity["code"].replace(".spold", "")
    #print(fn)
    try:
        index = codes.index(fn)
        #print(processors_filenames[index])
        activities.append((activity, *processors_data[index]))
        remaining.remove(processors_data[index])
    except ValueError:
        pass

100%|██████████| 21263/21263 [00:01<00:00, 14537.56it/s]


In [103]:
activities

[('heat, from municipal waste incineration to generic market for heat district or industrial, other than natural gas' (megajoule, PT, None),
  'chp_wte_back_pressure',
  'heat',
  '37ebaba3250e37c42d28a353f27632f5'),
 ('market for biomethane, high pressure' (cubic meter, CH, None),
  'biofuel_to_methane',
  'methane',
  '7d82a9b0a28dc30058260b5c68fbad95'),
 ('heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, PT, None),
  'chp_biofuel_extraction',
  'electricity',
  '1952fb157a18463b0028629917057bcf'),
 ('electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, PT, None),
  'roof_mounted_pv',
  'electricity',
  '1d7a3591e7a3d033a25e0eaf699bb50b'),
 ('electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, PT, None),
  'wind_onshore',
  'electricity',
  '81174ec2c20931c1a36f65c654bbd11e'),
 ('heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' 

In [104]:
# check the remaning ones (missing)
remaining

[('electrolysis', 'electricity', 'FILL'),
 ('existing_wind', 'electricity', '81174ec2c20931c1a36f65c654bbd11e'),
 ('existing_pv', 'electricity', '206c2e2c30f45d47ea2e9a3701f8ecc5'),
 ('waste_supply', 'electricity', 'fd6356e68720aec505c69cdadb3046fc'),
 ('biofuel_boiler', 'heat', '413bc4617794c6e8b07038dbeca64adb'),
 ('methane_boiler', 'heat', 'a3039e0a35ed472774d6ed66262c67f3')]

In [105]:
from collections import Counter

potential_aliases = [act[1] for act in activities]
print("All unique:", len(potential_aliases) == len(set(potential_aliases)))


All unique: False


In [106]:
# user a counter, and redo the array of activities, by adding a string the beginning with a new full unique (hopefully) alias
counts = Counter(potential_aliases)

activities_ = []
for act in activities:
    full_alias = f"{act[1]}_{act[2]}"
    activities_.append((full_alias, *act[1:]))

# check if its all good (should give False)
any((v > 1 for v in Counter([act[0] for act in activities_]).values()))

False

In [107]:
from ipywidgets import HTML
import tabulate

activities = activities_

table = tabulate.tabulate(activities, headers=["full alias [0]", "orig alias", "carrier", "code [3]"], tablefmt='html')
display(HTML(table))

HTML(value='<table>\n<thead>\n<tr><th>full alias [0]                    </th><th>orig alias            </th><t…

In [108]:
from bw2data.backends import Activity

activities_bw: dict[str, Activity] = {}
for activity in activities:
    activities_bw[activity[0]] = db.get_node(activity[3])

activities_bw

{'chp_wte_back_pressure_heat': 'heat, from municipal waste incineration to generic market for heat district or industrial, other than natural gas' (megajoule, PT, None),
 'biofuel_to_methane_methane': 'market for biomethane, high pressure' (cubic meter, CH, None),
 'chp_biofuel_extraction_electricity': 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, PT, None),
 'roof_mounted_pv_electricity': 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, PT, None),
 'wind_onshore_electricity': 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, PT, None),
 'chp_methane_extraction_heat': 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (megajoule, PT, None),
 'Heat_storage_big_heat': 'market for heat storage, 2000l' (unit, GLO, None),
 'hydro_run_of_river_electricity': 'electricity production, hydro, run-of-river' (kilowatt hour, PT, Non

In [119]:
SEEDS_DB_NAME = "seeds"
seeds_db = bw2data.Database(SEEDS_DB_NAME)
seeds_db.register()

In [120]:
# from enbios2.bw2.util import full_duplicate

def full_duplicate(activity: Activity, code=None, **kwargs) -> Activity:
    """
    Make a copy of an activity with its upstream exchanges
    (Otherwise, you cannot calculate the lca of the copy)
    :param activity: the activity to copy
    :param code: code of the new activity
    :param kwargs: other data for the copy
    :return: new activity
    """
    activity_copy = activity.copy(code, **kwargs)
    for upstream in activity.upstream():
        upstream.output.new_exchange(input=activity_copy, type=upstream["type"], amount=upstream.amount).save()
    activity_copy.save()
    return activity_copy


for alias, activity in tqdm(activities_bw.items()):
    clone1 = full_duplicate(activity)
    clone1["name"] = clone1["name"] + "__PRT_1"
    clone1["database"] = SEEDS_DB_NAME
    clone1["alias"] = alias + "__PRT_1"
    clone1.save()

    clone2 = full_duplicate(activity)
    clone2["name"] = clone2["name"] + "__PRT_2"
    clone2["database"] = SEEDS_DB_NAME
    clone2["alias"] = alias + "__PRT_2"
    clone2.save()

  0%|          | 0/23 [00:00<?, ?it/s]

Successfully switch activity dataset to database `seeds`


  4%|▍         | 1/23 [00:02<00:48,  2.18s/it]

Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`


  9%|▊         | 2/23 [00:07<01:28,  4.20s/it]

...

 91%|█████████▏| 21/23 [02:17<00:20, 10.13s/it]

Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`


 96%|█████████▌| 22/23 [02:22<00:08,  8.72s/it]

Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`


100%|██████████| 23/23 [02:31<00:00,  6.61s/it]


In [121]:
list(seeds_db)

['market for ethanol, without water, in 99.7% solution state, from fermentation, vehicle grade__PRT_2' (kilogram, CH, None),
 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical__PRT_2' (megajoule, PT, None),
 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014__PRT_2' (megajoule, PT, None),
 'electricity production, wind, 1-3MW turbine, onshore__PRT_2' (kilowatt hour, PT, None),
 'electricity production, hydro, pumped storage__PRT_1' (kilowatt hour, PT, None),
 'market for fatty acid methyl ester__PRT_2' (kilogram, RoW, None),
 'electricity production, hydro, reservoir, non-alpine region__PRT_1' (kilowatt hour, PT, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted__PRT_2' (kilowatt hour, PT, None),
 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si__PRT_2' (kilowatt hour, PT, None),
 'electricity production, photovoltaic, 570kWp open ground in

In [122]:
# note for ramin:
# clone1 where not moved to a new db, so cutoff_3.9.1_default has few duplicates... NICE!

In [123]:
enbios2_activities = {
    activity["alias"]: {
        "id": {"code": activity["code"]}
    }
    for activity in list(seeds_db)
}



In [124]:
enbios2_activities

{'roof_mounted_pv_electricity__PRT_1': {'id': {'code': '6a45fe6fd4e64c10ade3d3008906b15d'}},
 'chp_biofuel_extraction_electricity__PRT_2': {'id': {'code': 'e99b6058336a4a22b9b823953d1b2f4a'}},
 'heat_storage_small_heat__PRT_2': {'id': {'code': '34e94c8ac69346bab09d58c589bcae7a'}},
 'roof_mounted_pv_electricity__PRT_2': {'id': {'code': 'df1034c41121435cad7e894557b06419'}},
 'ccgt_electricity__PRT_2': {'id': {'code': '15c33229278544769187cf78a172afb4'}},
 'chp_methane_extraction_heat__PRT_2': {'id': {'code': '985be882592a46f3a981851e1f8583b0'}},
 'chp_methane_extraction_electricity__PRT_1': {'id': {'code': '58e1be85d76044a8886ef618cccfc54e'}},
 'Methane_storage_methane__PRT_1': {'id': {'code': '21940e0aab22486cb5e39eac9306121a'}},
 'chp_methane_extraction_electricity__PRT_2': {'id': {'code': '3387f9be47ac479ebd1d678e29cbe4fd'}},
 'pumped_hydro_electricity__PRT_1': {'id': {'code': '82ce0fc67f5a49968a227d1f3fa0da22'}},
 'chp_biofuel_extraction_electricity__PRT_1': {'id': {'code': '603a52d0

In [117]:
# in case things got messed up:
# seeds_db.delete_instance()

In [136]:
from csv import DictReader
from enbios2.generic.files import ReadDataPath

enbios2scenarios = []
raw_scenarios = ReadDataPath(base_folder / "data.csv") # .read_data()
raw_scenario_data = list(DictReader(raw_scenarios.open(encoding="utf-8"), delimiter=";"))
raw_scenario_data[:5]

[{'Scenarios': '0_',
  'year': '2012',
  'techs': 'battery',
  'locs': 'PRT_1',
  'carriers': 'electricity',
  'unit': 'twh',
  'flow_out_sum': '0.000525631'},
 {'Scenarios': '0_',
  'year': '2012',
  'techs': 'battery',
  'locs': 'PRT_2',
  'carriers': 'electricity',
  'unit': 'twh',
  'flow_out_sum': '0.000575126'},
 {'Scenarios': '0_',
  'year': '2012',
  'techs': 'biofuel_boiler',
  'locs': 'PRT_1',
  'carriers': 'heat',
  'unit': 'twh',
  'flow_out_sum': '8.9698237'},
 {'Scenarios': '0_',
  'year': '2012',
  'techs': 'biofuel_boiler',
  'locs': 'PRT_2',
  'carriers': 'heat',
  'unit': 'twh',
  'flow_out_sum': '0.000386907'},
 {'Scenarios': '0_',
  'year': '2012',
  'techs': 'biofuel_supply',
  'locs': 'PRT_1',
  'carriers': 'biofuel',
  'unit': 'twh',
  'flow_out_sum': '20.39066515'}]

In [144]:
current_scenario_index = 0
current_scenario = {}
for row in raw_scenario_data:
    scenario_index = int(row["Scenarios"][:-1])
    assert current_scenario_index <= scenario_index <= current_scenario_index + 1
    if scenario_index == current_scenario_index + 1:
        current_scenario_index = scenario_index
        enbios2scenarios.append(current_scenario.copy())
        current_scenario = {}



0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
0 0
0
1 0
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
1 1
1
2 1
2
2 2
2
2 2


In [132]:
from ast import literal_eval

methods = workbook["ScalarIndicators"]

enbios2_methods = {

}

for row in methods.iter_rows(min_row=2):
    if row[5].value:
        method_tuple = literal_eval(row[5].value)
        enbios2_methods[method_tuple[2]] = {
            "id": method_tuple
        }

enbios2_methods

{'ozone depletion potential (ODPinfinite)': {'id': ('ReCiPe 2016 v1.03, midpoint (H)',
   'ozone depletion',
   'ozone depletion potential (ODPinfinite)')},
 'agricultural land occupation (LOP)': {'id': ('ReCiPe 2016 v1.03, midpoint (H)',
   'land use',
   'agricultural land occupation (LOP)')},
 'surplus ore potential (SOP)': {'id': ('ReCiPe 2016 v1.03, midpoint (H)',
   'material resources: metals/minerals',
   'surplus ore potential (SOP)')},
 'global warming potential (GWP1000)': {'id': ('ReCiPe 2016 v1.03, midpoint (H)',
   'climate change',
   'global warming potential (GWP1000)')}}

In [ ]:
enbios2_data = {
    "activities": enbios2_activities,
    "methods": enbios2_methods
}